# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).


In [ ]:
pip install xlsxwriter

In [ ]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from google.colab import files

## Importing Our List of Stocks & API Token


In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretly import IEX_CLOUD_API_TOKEN

## Executing A Batch API Call & Building Our DataFrame



In [ ]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio']
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio
0,A,162.15,49.69
1,AAL,20.19,-3.96
2,AAP,230.29,25.62
3,AAPL,149.38,30.25
4,ABBV,112.20,29.59
...,...,...,...
500,YUM,131.13,29.37
501,ZBH,152.59,35.2
502,ZBRA,540.45,38.93
503,ZION,67.42,6.68


## Removing Glamour Stocks


In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio
0,EBAY,81.89,4.55
1,BIO,796.31,5.81
2,MPC,70.40,5.88
3,PRU,117.07,6.37
4,ZION,67.42,6.68
5,GM,60.25,6.82
6,COF,169.71,6.93
7,AFL,59.01,6.96
8,SYF,49.51,6.98
9,CINF,123.50,7.06


## Building a Better (and More Realistic) Value Strategy



In [ ]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [ ]:
rv_columns = [
    'Ticker',
    'Price',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame



In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,190.00,60.53,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,334.79,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,298.41,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.00,10.21,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,24.90,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.73,14.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.01,32.06,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.76,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2884.62,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,107.64,35.32,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


We will replace missing data with the average non-`NaN` data point from that column. 


In [ ]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [ ]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

In [ ]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0       0.833663
1      0.0712871
2       0.469307
3            0.6
4       0.568317
         ...    
500     0.560396
501     0.689109
502     0.710891
503    0.0891089
504     0.851485
Name: PE Percentile, Length: 505, dtype: object
0        0.79703
1       0.049505
2       0.557426
3       0.960396
4       0.885149
         ...    
500    0.0455446
501     0.326733
502     0.821782
503     0.135644
504     0.934653
Name: PB Percentile, Length: 505, dtype: object
0       0.811881
1      0.0356436
2       0.173267
3       0.792079
4       0.521782
         ...    
500     0.740594
501     0.570297
502     0.683168
503     0.452475
504     0.914851
Name: PS Percentile, Length: 505, dtype: object
0       0.883168
1      0.0277228
2       0.372277
3       0.742574
4       0.277228
         ...    
500      0.69505
501     0.683168
502     0.831683
503    0.0554455
504     0.910891
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.825743
1      0.0594059
2       0.138614
3 

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,159.27,50.66,0.833663,9.87,0.79703,7.9800,0.811881,32.540248,0.883168,14.984284,0.825743,N/A
1,AAL,20.19,-3.83,0.0712871,-1.69,0.049505,0.5142,0.0356436,-7.281999,0.0277228,1.507409,0.0594059,N/A
2,AAP,237.25,25.08,0.469307,4.62,0.557426,1.3800,0.173267,12.839341,0.372277,3.062687,0.138614,N/A
3,AAPL,155.31,30.54,0.6,38.35,0.960396,7.3400,0.792079,23.626193,0.742574,18.467437,0.893069,N/A
4,ABBV,109.70,29.39,0.568317,15.70,0.885149,3.7400,0.521782,10.613457,0.277228,7.345826,0.463366,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.84,28.96,0.560396,-4.77,0.0455446,6.1500,0.740594,20.803502,0.69505,10.210325,0.659406,N/A
501,ZBH,158.04,35.70,0.689109,2.50,0.326733,4.1800,0.570297,20.637369,0.683168,6.849213,0.423762,N/A
502,ZBRA,532.91,37.93,0.710891,11.22,0.821782,5.7200,0.683168,28.241065,0.831683,11.898066,0.736634,N/A
503,ZION,67.09,6.58,0.0891089,1.46,0.135644,3.1900,0.452475,5.220272,0.0554455,2.776669,0.124752,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

In [ ]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)*10
    
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,159.27,50.66,0.833663,9.87,0.79703,7.9800,0.811881,32.540248,0.883168,14.984284,0.825743,8.30297
1,AAL,20.19,-3.83,0.0712871,-1.69,0.049505,0.5142,0.0356436,-7.281999,0.0277228,1.507409,0.0594059,0.487129
2,AAP,237.25,25.08,0.469307,4.62,0.557426,1.3800,0.173267,12.839341,0.372277,3.062687,0.138614,3.42178
3,AAPL,155.31,30.54,0.6,38.35,0.960396,7.3400,0.792079,23.626193,0.742574,18.467437,0.893069,7.97624
4,ABBV,109.70,29.39,0.568317,15.70,0.885149,3.7400,0.521782,10.613457,0.277228,7.345826,0.463366,5.43168
...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,128.84,28.96,0.560396,-4.77,0.0455446,6.1500,0.740594,20.803502,0.69505,10.210325,0.659406,5.40198
501,ZBH,158.04,35.70,0.689109,2.50,0.326733,4.1800,0.570297,20.637369,0.683168,6.849213,0.423762,5.38614
502,ZBRA,532.91,37.93,0.710891,11.22,0.821782,5.7200,0.683168,28.241065,0.831683,11.898066,0.736634,7.56832
503,ZION,67.09,6.58,0.0891089,1.46,0.135644,3.1900,0.452475,5.220272,0.0554455,2.776669,0.124752,1.71485


## Formatting Our Excel Output



In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Price-to-Earnings Ratio', float_template],
                    'D': ['PE Percentile', percent_template],
                    'E': ['Price-to-Book Ratio', float_template],
                    'F': ['PB Percentile',percent_template],
                    'G': ['Price-to-Sales Ratio', float_template],
                    'H': ['PS Percentile', percent_template],
                    'I': ['EV/EBITDA', float_template],
                    'J': ['EV/EBITDA Percentile', percent_template],
                    'K': ['EV/GP', float_template],
                    'L': ['EV/GP Percentile', percent_template],
                    'M': ['RV', integer_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()
files.download('value_strategy.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>